<a href="https://colab.research.google.com/github/mkufel/CCA2/blob/master/german_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install keras-bert
!pip install pyspellchecker
# !pip install tempfile

import os

pretrained_path = '/content/drive/My Drive/Commerzbank/bert-base-en-uncased'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

"""
pretrained_path = '/content/drive/My Drive/Master/bert-base-german-cased'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert-base-german-cased')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
"""
seq_len=256
#os.environ['TF_KERAS'] = '1'

In [0]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict, cased=False)

Using TensorFlow backend.


In [0]:
import tensorflow as tf
import numpy as np
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

In [0]:
from keras_bert import load_trained_model_from_checkpoint
model = load_trained_model_from_checkpoint(config_path, checkpoint_path,training=True,trainable=True,seq_len=seq_len)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
import re
import pandas as pd

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

train, test = download_and_load_datasets()

train = train.sample(25000,random_state=42)
test = test.sample(25000,random_state=42)
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

84131840/84125825 [==============================] - 9s 0us/step


In [0]:
indices=[]
segments=[]
masks=[]
for inst in train.iloc[:,0].values:
  tokens = tokenizer.tokenize(inst)
  idx, seg = tokenizer.encode(first=inst, max_len=seq_len)
  indices.append(idx)
  segments.append(seg)
  masks.append([1] * (min(len(tokens)+2,seq_len))+[0]*(max(seq_len-(len(tokens)+2),0)))

t_indices=[]
t_segments=[]
t_masks=[]
for inst in test.iloc[:1000,0].values:
  tokens = tokenizer.tokenize(inst)
  idx, seg = tokenizer.encode(first=inst, max_len=seq_len)
  t_indices.append(idx)
  t_segments.append(seg)
  t_masks.append([1] * (min(len(tokens)+2,seq_len))+[0]*(max(seq_len-(len(tokens)+2),0)))

In [0]:
from keras.models import Model
from keras.layers import Lambda
from keras.layers import Dense
from keras.layers import concatenate, Input
from keras_bert.layers import MaskedGlobalMaxPool1D

#tokens=tra_tokenizer.tokenize(text)
import numpy as np
#input_ids, segment_ids=tokenizer.encode_2(first=tokens,max_len=256)
#input_mask = [1] * (len(tokens)+2)+[0]*(max_len-(len(tokens)+2))
layer_size=768

amount=Input(shape=(1,),name='Amount')
pooled = Lambda( lambda x: tf.slice(x, [0,0,0], [tf.shape(model.output[0])[0],1,layer_size]))(model.get_layer('Encoder-4-FeedForward-Add').output)
pooled = Lambda( lambda x: tf.reshape(x,[tf.shape(model.output[0])[0],layer_size]))(pooled)
#conc=concatenate([pooled, amount], axis=1)
dense = Dense(256,activation=tf.nn.relu,trainable=True)(pooled)
#pred = Dense(64,activation=tf.nn.softmax)(dense)
pred = Dense(1,activation=tf.nn.sigmoid,trainable=True)(dense)


#dense=tf.compat.v1.layers.dense(pooled, 256, activation=tf.nn.relu)
#dense = Lambda( lambda x: tf.layers.dense(x,256,tf.nn.relu))(pooled)
#pred = Lambda( lambda x: tf.layers.dense(x,64,tf.nn.softmax))(dense)
#print(len(model.predict([np.array([input_ids]),np.array([segment_ids]),np.array([input_mask])])[0][0][0]))
#pooled=tf.keras.backend.slice(,[0,0,0],)
#pooled=
#model.output
#dense = tf.keras.layers.Dense(256, activation='relu')(pooled)
#pred = tf.keras.layers.Dense(64, activation='softmax')(dense)  
#pred
model_2 = Model(inputs=[model.input[0],model.input[1],model.input[2]], outputs=pred)
print(model_2.input)
from keras.optimizers import adam
opt = adam(lr=1e-5)
model_2.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#model_2.summary()

[<tf.Tensor 'Input-Token:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'Input-Segment:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'Input-Masked:0' shape=(?, 256) dtype=float32>]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
print(len(masks[-1]))
print(np.array(masks).shape)
model_2.fit([np.array(indices),np.array(segments),np.array(masks)], train.iloc[:,2].values,validation_data=([np.array(t_indices), np.array(t_segments), np.array(t_masks)], test.iloc[:1000,2].values), epochs=15, batch_size=32)

256
(25000, 256)


Train on 25000 samples, validate on 1000 samples
Epoch 1/15


InvalidArgumentError: ignored

In [0]:
Epoch 1/15
25000/25000 [==============================] - 57s 2ms/step - loss: 0.5085 - acc: 0.7453 - val_loss: 0.4293 - val_acc: 0.7940
Epoch 2/15
25000/25000 [==============================] - 52s 2ms/step - loss: 0.4037 - acc: 0.8176 - val_loss: 0.4048 - val_acc: 0.8230
Epoch 3/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3619 - acc: 0.8392 - val_loss: 0.4046 - val_acc: 0.8220
Epoch 4/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.3280 - acc: 0.8586 - val_loss: 0.4060 - val_acc: 0.8220
Epoch 5/15
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2984 - acc: 0.8732 - val_loss: 0.4162 - val_acc: 0.8230
Epoch 6/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.2696 - acc: 0.8878 - val_loss: 0.4256 - val_acc: 0.8180
Epoch 7/15
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2442 - acc: 0.8992 - val_loss: 0.4385 - val_acc: 0.8260
Epoch 8/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.2184 - acc: 0.9130 - val_loss: 0.4836 - val_acc: 0.8120
Epoch 9/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.1942 - acc: 0.9243 - val_loss: 0.4986 - val_acc: 0.8200
Epoch 10/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.1681 - acc: 0.9352 - val_loss: 0.5184 - val_acc: 0.8260
Epoch 11/15
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1448 - acc: 0.9456 - val_loss: 0.5621 - val_acc: 0.8250
Epoch 12/15
25000/25000 [==============================] - 52s 2ms/step - loss: 0.1256 - acc: 0.9528 - val_loss: 0.5630 - val_acc: 0.8310
Epoch 13/15
25000/25000 [==============================] - 53s 2ms/step - loss: 0.1091 - acc: 0.9593 - val_loss: 0.6327 - val_acc: 0.8200
Epoch 14/15
25000/25000 [==============================] - 54s 2ms/step - loss: 0.0919 - acc: 0.9669 - val_loss: 0.6528 - val_acc: 0.8210
Epoch 15/15
25000/25000 [==============================] - 54s 2ms/step - loss: 0.0793 - acc: 0.9699 - val_loss: 0.7172 - val_acc: 0.8220





256
(25000, 256)
Train on 25000 samples, validate on 1000 samples
Epoch 1/15
25000/25000 [==============================] - 103s 4ms/step - loss: 0.4361 - acc: 0.7974 - val_loss: 0.3441 - val_acc: 0.8570
Epoch 2/15
25000/25000 [==============================] - 99s 4ms/step - loss: 0.3254 - acc: 0.8594 - val_loss: 0.3093 - val_acc: 0.8790
Epoch 3/15
25000/25000 [==============================] - 99s 4ms/step - loss: 0.2846 - acc: 0.8824 - val_loss: 0.2972 - val_acc: 0.8830
Epoch 4/15
25000/25000 [==============================] - 99s 4ms/step - loss: 0.2527 - acc: 0.8974 - val_loss: 0.3003 - val_acc: 0.8860
Epoch 5/15
25000/25000 [==============================] - 99s 4ms/step - loss: 0.2227 - acc: 0.9101 - val_loss: 0.3085 - val_acc: 0.8930
Epoch 6/15
25000/25000 [==============================] - 99s 4ms/step - loss: 0.2018 - acc: 0.9201 - val_loss: 0.3099 - val_acc: 0.8910
Epoch 7/15
25000/25000 [==============================] - 100s 4ms/step - loss: 0.1743 - acc: 0.9338 - val_loss: 0.3377 - val_acc: 0.8790
Epoch 8/15
 3776/25000 [===>..........................] - ETA: 1:22 - loss: 


 Epoch 1/15
25000/25000 [==============================] - 502s 20ms/step - loss: 0.3985 - acc: 0.8144 - val_loss: 0.2768 - val_acc: 0.8910
Epoch 2/15
25000/25000 [==============================] - 493s 20ms/step - loss: 0.2617 - acc: 0.8919 - val_loss: 0.2800 - val_acc: 0.8920
Epoch 3/15
25000/25000 [==============================] - 493s 20ms/step - loss: 0.2065 - acc: 0.9181 - val_loss: 0.2591 - val_acc: 0.9060
Epoch 4/15
25000/25000 [==============================] - 493s 20ms/step - loss: 0.1510 - acc: 0.9414 - val_loss: 0.2824 - val_acc: 0.9000
Epoch 5/15
25000/25000 [==============================] - 493s 20ms/step - loss: 0.0988 - acc: 0.9648 - val_loss: 0.3315 - val_acc: 0.8990
Epoch 6/15
25000/25000 [==============================] - 493s 20ms/step - loss: 0.0579 - acc: 0.9804 - val_loss: 0.3873 - val_acc: 0.8880
Epoch 7/15
 1088/25000 [>.............................] - ETA: 7:39 - loss: 0.0323 - acc: 0.9890
